In [2]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
import tensorflow.data as tf_data
import tensorflow.strings as tf_strings

import keras
from keras import layers
#from keras import ops
from keras.layers import TextVectorization

from sklearn.metrics import accuracy_score
from google.colab import drive as dr
dr.mount('/content/drive')

Mounted at /content/drive


In [3]:
barnette_graphs=np.load('drive/My Drive/Minor Work/Datasets/barnette_graph_data_alpha.npy',allow_pickle=True)

In [ ]:
barnette_graphs[2]

'13.14.15.16 5.6.2.1 2.6.8.4.10.9 3.4.8.7 5.1.12.11.3.7 5.6.8.7 10.4.3.11.14.13 12.1.2.9.16.15 14.11.12.15 16.9.10.13'

In [4]:

hamiltonian_cycles=np.load('drive/My Drive/Minor Work/Datasets/hamiltonian_data_alpha.npy',allow_pickle=True)

In [ ]:
hamiltonian_cycles[2]

array([1., 0., 1., 0., 1., 0., 1., 0., 0., 0.])

In [ ]:
len(barnette_graphs)

50000

In [ ]:
len(hamiltonian_cycles)

50000

In [5]:
max_barnette_len=0
for j in range(len(barnette_graphs)):
    if len(barnette_graphs[j])>max_barnette_len:
        max_barnette_len=len(barnette_graphs[j])

In [ ]:
max_barnette_len

368

In [6]:
barnette_vocab=[
'0',
'1',
'2',
'3',
'4',
'5',
'6',
'7',
'8',
'9',
' ',
'.',
'p',
''
]

In [7]:
barnette_vocab_dict={}
for i in range(len(barnette_vocab)):
    barnette_vocab_dict[barnette_vocab[i]]=i

In [8]:
barnette_vocab_rev_dict={}
for i in range(len(barnette_vocab)):
    barnette_vocab_rev_dict[i]=barnette_vocab[i]

In [ ]:
barnette_vocab_dict

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 ' ': 10,
 '.': 11,
 'p': 12,
 '': 13}

In [9]:
def text_to_numeric(data,vocab_dict,max_len):
    data_numeric=[]
    for i in range(len(data)):
        numeric=[]

        for char in data[i]:
            numeric.append(vocab_dict[char])

        count=len(numeric)

        while count<=max_len:
            numeric.append(vocab_dict['p'])
            count=count+1
        data_numeric.append(np.array(numeric))

    return np.array(data_numeric)

In [10]:
barnette_graphs_numeric=text_to_numeric(barnette_graphs,barnette_vocab_dict, 400)

In [ ]:
barnette_graphs_numeric.shape

(50000, 401)

In [ ]:
barnette_graphs_numeric[2]

array([ 1,  3, 11,  1,  4, 11,  1,  5, 11,  1,  6, 10,  5, 11,  6, 11,  2,
       11,  1, 10,  2, 11,  6, 11,  8, 11,  4, 11,  1,  0, 11,  9, 10,  3,
       11,  4, 11,  8, 11,  7, 10,  5, 11,  1, 11,  1,  2, 11,  1,  1, 11,
        3, 11,  7, 10,  5, 11,  6, 11,  8, 11,  7, 10,  1,  0, 11,  4, 11,
        3, 11,  1,  1, 11,  1,  4, 11,  1,  3, 10,  1,  2, 11,  1, 11,  2,
       11,  9, 11,  1,  6, 11,  1,  5, 10,  1,  4, 11,  1,  1, 11,  1,  2,
       11,  1,  5, 10,  1,  6, 11,  9, 11,  1,  0, 11,  1,  3, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 12, 12

In [11]:
max_hamiltonian_len=0
for i in range(len(hamiltonian_cycles)):
    if len(hamiltonian_cycles[i])>max_hamiltonian_len:
        max_hamiltonian_len=len(hamiltonian_cycles[i])

In [12]:
#hamiltonian_cycles_padded=np.zeros((len(hamiltonian_cycles),max_hamiltonian_len))-1
hamiltonian_cycles_padded=np.zeros((len(hamiltonian_cycles),max_hamiltonian_len))
for i in range(len(hamiltonian_cycles)):
    for j in range(len(hamiltonian_cycles[i])):
        hamiltonian_cycles_padded[i][j]=hamiltonian_cycles[i][j]

In [ ]:
hamiltonian_cycles_padded[2]

array([1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

In [13]:
num_train_samples = int(0.8 * len(hamiltonian_cycles))

barnette_graphs_trn = barnette_graphs_numeric[:num_train_samples]
hamiltonian_cycles_trn = hamiltonian_cycles_padded[:num_train_samples]

barnette_graphs_tst = barnette_graphs_numeric[num_train_samples:]
hamiltonian_cycles_tst = hamiltonian_cycles_padded[num_train_samples:]


print(f"{len(hamiltonian_cycles)} total samples")
print(f"{len(hamiltonian_cycles_trn)} training samples")
print(f"{len(hamiltonian_cycles_tst)} test samples")

50000 total samples
40000 training samples
10000 test samples


In [ ]:
barnette_graphs_trn.shape

(40000, 401)

In [ ]:
hamiltonian_cycles_trn.shape

(40000, 24)

In [ ]:
def cv_data_split(barnette_graphs_numeric, hamiltonian_cycles_numeric, cv=10):


    data_length=len(barnette_graphs_numeric)
    r=int(data_length/cv)
    division_ranges=(np.arange(cv))*r
    folds_data=[]
    folds_labels=[]
    for i in range(cv-1):
        starting_point=int(division_ranges[i])
        ending_point=int(division_ranges[i+1])
        folds_data.append(barnette_graphs_numeric[starting_point:ending_point])
        folds_labels.append(hamiltonian_cycles_numeric[starting_point:ending_point])
    folds_data.append(barnette_graphs_numeric[ending_point:])
    folds_labels.append(hamiltonian_cycles_numeric[ending_point:])

    training_folds_data=[]
    training_folds_labels=[]
    for i in range(cv):
        training_folds_data_temp=[]
        training_folds_labels_temp=[]
        for j in range(cv):
            if j!=i:
                for k in range(len(folds_data[j])):
                    training_folds_data_temp.append(folds_data[j][k])
                    training_folds_labels_temp.append(folds_labels[j][k])
        training_folds_data.append(training_folds_data_temp)
        training_folds_labels.append(training_folds_labels_temp)

    testing_folds_data=folds_data
    testing_folds_labels=folds_labels
    return np.array(training_folds_data), np.array(training_folds_labels), np.array(testing_folds_data), np.array(testing_folds_labels)

In [ ]:
training_folds_data, training_folds_labels, testing_folds_data, testing_folds_labels =cv_data_split(barnette_graphs_trn, hamiltonian_cycles_trn, cv=10)

In [ ]:
training_folds_data.shape

(10, 36000, 401)

In [ ]:
np.save('Training_data', training_folds_data)
np.save('Training_label', training_folds_labels)
np.save('Testing_data', testing_folds_data)
np.save('Testing_label', testing_folds_labels)

In [ ]:
training_folds_labels.shape

(10, 36000, 24)

In [ ]:
testing_folds_data.shape

(10, 4000, 401)

In [ ]:
testing_folds_labels.shape

(10, 4000, 24)

In [ ]:
class Barnette_Graph(object):
    def __init__(self, face_vertex_dict):

        # states can be defined as path connected subsets of the set of faces such that,
            # 1. have no points in thier interior
            # 2. the complement of the set of faces contained in a state is also path connected

        # face_vertex_dict is a dictionary whose keys are faces and values are the vertex indices of the corresponding key face
        # we input

        self.face_vertex_dict=face_vertex_dict

    def string_to_list(self,string):
        l=string.split('.')
        return(l)

    def vertices_in_state(self, state):
        state_list=self.string_to_list(state)
        vertices_in_state=[]
        for face in state_list:
            vertices_in_face=self.face_vertex_dict[face]
            for vertex in vertices_in_face:
                vertices_in_state.append(vertex)
        return vertices_in_state

    def boundary_and_interior(self, state):
        vertices_in_state=self.vertices_in_state(state)
        unique, counts = np.unique(np.array(vertices_in_state), return_counts=True)
        boundary_vertex_list=unique[counts<3]
        interior_vertex_list=unique[counts>=3]
        return boundary_vertex_list, interior_vertex_list

    def nebs_of_face(self, face):
        face_vertices=set(self.face_vertex_dict[face])
        nebs=[]
        for key in self.face_vertex_dict.keys():
            neb_vertices=set(self.face_vertex_dict[key])
            if len(face_vertices.intersection(neb_vertices))==2:
                nebs.append(key)
        return nebs

    def is_valid_state(self,state,face):
        ## by construction a state is path connected
        ## check how many vertices in the interior
        newState=state+'.'+face
        _,interior_vertices=self.boundary_and_interior(newState)


        ## check whether complement is path connected
        vertices_in_state=self.vertices_in_state(state)
        state_vertices=set(vertices_in_state)
        face_vertices=set(self.face_vertex_dict[face])

        if len(face_vertices.intersection(state_vertices))>2:
            return False
        elif len(interior_vertices)>0:
            return False
        else:
            return True

        return (succ)

In [14]:

#import keras.ops as ops

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(dense_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:,tf.newaxis, tf.newaxis, :], dtype='int32')
            #ops.cast(mask[:, None, :], dtype="int32")
        else:
            padding_mask = None

        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "dense_dim": self.dense_dim,
                "num_heads": self.num_heads,
            }
        )
        return config


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        #ops.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        #ops.arange(0, length, 1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        if mask is None:
            return None
        else:
            return tf.math.not_equal(inputs,0)
        #ops.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [
                layers.Dense(latent_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, None, :], dtype="int32")
            #ops.cast(mask[:, None, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
            #ops.minimum(padding_mask, causal_mask)
        else:
            padding_mask = None

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        #ops.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, None]
        #ops.arange(sequence_length)[:, None]
        j = tf.range(sequence_length)
        #ops.arange(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        #ops.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        #ops.reshape(mask, (1, input_shape[1], input_shape[1]))
        #mult = ops.concatenate(
        #    [ops.expand_dims(batch_size, -1), ops.convert_to_tensor([1, 1])],
        #    axis=0,
        #)
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.convert_to_tensor([1, 1])],
            axis=0,
        )
        return tf.tile(mask, mult)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "embed_dim": self.embed_dim,
                "latent_dim": self.latent_dim,
                "num_heads": self.num_heads,
            }
        )
        return config


In [15]:
embed_dim = 256
latent_dim = 2048
num_heads = 2

barnette_graph = keras.Input(shape=(barnette_graphs_trn.shape[1],), dtype="int64")
x_1 = PositionalEmbedding(2*max_barnette_len, len(barnette_vocab), embed_dim)(barnette_graph)
x_2 = TransformerEncoder(embed_dim, latent_dim, num_heads)(x_1)
x_3 = keras.layers.Flatten()(x_2)
x_4 = keras.layers.Dense(132, activation='relu')(x_3)
hamiltonian_cycle = keras.layers.Dense(hamiltonian_cycles_padded[0].shape[0], activation='sigmoid')(x_4)
model = keras.Model(inputs=barnette_graph, outputs=hamiltonian_cycle)

In [16]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 401)]             0         
                                                                 
 positional_embedding (Posi  (None, 401, 256)          192000    
 tionalEmbedding)                                                
                                                                 
 transformer_encoder (Trans  (None, 401, 256)          1577984   
 formerEncoder)                                                  
                                                                 
 flatten (Flatten)           (None, 102656)            0         
                                                                 
 dense_2 (Dense)             (None, 132)               13550724  
                                                                 
 dense_3 (Dense)             (None, 24)                3192  

In [17]:
X_train, Y_train, X_test, Y_test = np.load('drive/My Drive/Minor Work/Datasets/Modified CV/Training_data.npy'), np.load('drive/My Drive/Minor Work/Datasets/Modified CV/Training_label.npy'), np.load('drive/My Drive/Minor Work/Datasets/Modified CV/Testing_data.npy'), np.load('drive/My Drive/Minor Work/Datasets/Modified CV/Testing_label.npy')

In [18]:
#1st fold
epochs = 30 # This should be at least 30 for convergence

fold_wise_binary_accuracy=[]
fold_wise_exact_match=[]

#for i in range(1):
for i in range(0, 1):
    model.compile(keras.optimizers.Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["binary_accuracy"])

    model.fit(X_train[i],Y_train[i], epochs= epochs, validation_split=0.1)

    print('Model training done for fold '+str(i))
    predictions=model.predict(X_test[i])
    print('Predictions done for fold '+str(i))

    bins = np.array([0.5])
    count_exact_match=0
    sum_acc=0

    for j in range(len(predictions)):
        discrete_prediction=np.digitize(predictions[j], bins)
        acc=accuracy_score(Y_test[i][j],discrete_prediction)
        sum_acc=sum_acc+acc
        if acc==1:
            count_exact_match=count_exact_match+1
    avg_acc=sum_acc/len(predictions)
    exact_match_ratio=count_exact_match/len(predictions)

    fold_wise_binary_accuracy.append(avg_acc)
    fold_wise_exact_match.append(exact_match_ratio)

    print('for fold '+ str(i)+' binary accuracy is: ', avg_acc)
    print('for fold '+ str(i)+' exact match ratio is: ', exact_match_ratio)


Epoch 1/30
1013/1013 [==============================] - 70s 63ms/step - loss: 0.1655 - binary_accuracy: 0.9317 - val_loss: 0.1236 - val_binary_accuracy: 0.9472
Epoch 2/30
1013/1013 [==============================] - 65s 64ms/step - loss: 0.1064 - binary_accuracy: 0.9553 - val_loss: 0.1078 - val_binary_accuracy: 0.9539
Epoch 3/30
1013/1013 [==============================] - 66s 65ms/step - loss: 0.0882 - binary_accuracy: 0.9628 - val_loss: 0.1017 - val_binary_accuracy: 0.9583
Epoch 4/30
1013/1013 [==============================] - 67s 66ms/step - loss: 0.0763 - binary_accuracy: 0.9680 - val_loss: 0.0962 - val_binary_accuracy: 0.9614
Epoch 5/30
1013/1013 [==============================] - 71s 70ms/step - loss: 0.0681 - binary_accuracy: 0.9715 - val_loss: 0.0923 - val_binary_accuracy: 0.9634
Epoch 6/30
1013/1013 [==============================] - 68s 67ms/step - loss: 0.0609 - binary_accuracy: 0.9747 - val_loss: 0.0905 - val_binary_accuracy: 0.9652
Epoch 7/30
1013/1013 [==================

In [ ]:
#3rd fold
epochs = 30 # This should be at least 30 for convergence

fold_wise_binary_accuracy=[]
fold_wise_exact_match=[]

#for i in range(1):
for i in range(2, 3):
    model.compile(keras.optimizers.Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["binary_accuracy"])

    model.fit(X_train[i],Y_train[i], epochs= epochs, validation_split=0.1)

    print('Model training done for fold '+str(i))
    predictions=model.predict(X_test[i])
    print('Predictions done for fold '+str(i))

    bins = np.array([0.5])
    count_exact_match=0
    sum_acc=0

    for j in range(len(predictions)):
        discrete_prediction=np.digitize(predictions[j], bins)
        acc=accuracy_score(Y_test[i][j],discrete_prediction)
        sum_acc=sum_acc+acc
        if acc==1:
            count_exact_match=count_exact_match+1
    avg_acc=sum_acc/len(predictions)
    exact_match_ratio=count_exact_match/len(predictions)

    fold_wise_binary_accuracy.append(avg_acc)
    fold_wise_exact_match.append(exact_match_ratio)

    print('for fold '+ str(i)+' binary accuracy is: ', avg_acc)
    print('for fold '+ str(i)+' exact match ratio is: ', exact_match_ratio)


Epoch 1/30
1013/1013 [==============================] - 77s 71ms/step - loss: 0.1713 - binary_accuracy: 0.9279 - val_loss: 0.1286 - val_binary_accuracy: 0.9459
Epoch 2/30
1013/1013 [==============================] - 70s 70ms/step - loss: 0.1092 - binary_accuracy: 0.9536 - val_loss: 0.1057 - val_binary_accuracy: 0.9554
Epoch 3/30
1013/1013 [==============================] - 73s 72ms/step - loss: 0.0895 - binary_accuracy: 0.9622 - val_loss: 0.0991 - val_binary_accuracy: 0.9583
Epoch 4/30
1013/1013 [==============================] - 71s 70ms/step - loss: 0.0776 - binary_accuracy: 0.9674 - val_loss: 0.0965 - val_binary_accuracy: 0.9604
Epoch 5/30
1013/1013 [==============================] - 73s 72ms/step - loss: 0.0692 - binary_accuracy: 0.9710 - val_loss: 0.0920 - val_binary_accuracy: 0.9628
Epoch 6/30
1013/1013 [==============================] - 71s 70ms/step - loss: 0.0627 - binary_accuracy: 0.9739 - val_loss: 0.0919 - val_binary_accuracy: 0.9643
Epoch 7/30
1013/1013 [==================

In [ ]:
#Re run the below code

In [ ]:
#5th fold
epochs = 30 # This should be at least 30 for convergence

fold_wise_binary_accuracy=[]
fold_wise_exact_match=[]

#for i in range(1):
for i in range(4, 5):
    model.compile(keras.optimizers.Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["binary_accuracy"])

    model.fit(X_train[i],Y_train[i], epochs= epochs, validation_split=0.1)

    print('Model training done for fold '+str(i))
    predictions=model.predict(X_test[i])
    print('Predictions done for fold '+str(i))

    bins = np.array([0.5])
    count_exact_match=0
    sum_acc=0

    for j in range(len(predictions)):
        discrete_prediction=np.digitize(predictions[j], bins)
        acc=accuracy_score(Y_test[i][j],discrete_prediction)
        sum_acc=sum_acc+acc
        if acc==1:
            count_exact_match=count_exact_match+1
    avg_acc=sum_acc/len(predictions)
    exact_match_ratio=count_exact_match/len(predictions)

    fold_wise_binary_accuracy.append(avg_acc)
    fold_wise_exact_match.append(exact_match_ratio)

    print('for fold '+ str(i)+' binary accuracy is: ', avg_acc)
    print('for fold '+ str(i)+' exact match ratio is: ', exact_match_ratio)


Epoch 1/30
1013/1013 [==============================] - 80s 72ms/step - loss: 0.0241 - binary_accuracy: 0.9902 - val_loss: 0.1031 - val_binary_accuracy: 0.9720
Epoch 2/30
1013/1013 [==============================] - 73s 72ms/step - loss: 0.0226 - binary_accuracy: 0.9907 - val_loss: 0.1034 - val_binary_accuracy: 0.9721
Epoch 3/30
1013/1013 [==============================] - 75s 74ms/step - loss: 0.0213 - binary_accuracy: 0.9913 - val_loss: 0.1061 - val_binary_accuracy: 0.9725
Epoch 4/30
1013/1013 [==============================] - 73s 72ms/step - loss: 0.0205 - binary_accuracy: 0.9914 - val_loss: 0.1085 - val_binary_accuracy: 0.9723
Epoch 5/30
1013/1013 [==============================] - 75s 74ms/step - loss: 0.0197 - binary_accuracy: 0.9920 - val_loss: 0.1072 - val_binary_accuracy: 0.9729
Epoch 6/30
1013/1013 [==============================] - 75s 74ms/step - loss: 0.0194 - binary_accuracy: 0.9922 - val_loss: 0.1121 - val_binary_accuracy: 0.9727
Epoch 7/30
1013/1013 [==================

In [ ]:
#7th fold
epochs = 30 # This should be at least 30 for convergence

fold_wise_binary_accuracy=[]
fold_wise_exact_match=[]

#for i in range(1):
for i in range(6, 7):
    model.compile(keras.optimizers.Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["binary_accuracy"])

    model.fit(X_train[i],Y_train[i], epochs= epochs, validation_split=0.1)

    print('Model training done for fold '+str(i))
    predictions=model.predict(X_test[i])
    print('Predictions done for fold '+str(i))

    bins = np.array([0.5])
    count_exact_match=0
    sum_acc=0

    for j in range(len(predictions)):
        discrete_prediction=np.digitize(predictions[j], bins)
        acc=accuracy_score(Y_test[i][j],discrete_prediction)
        sum_acc=sum_acc+acc
        if acc==1:
            count_exact_match=count_exact_match+1
    avg_acc=sum_acc/len(predictions)
    exact_match_ratio=count_exact_match/len(predictions)

    fold_wise_binary_accuracy.append(avg_acc)
    fold_wise_exact_match.append(exact_match_ratio)

    print('for fold '+ str(i)+' binary accuracy is: ', avg_acc)
    print('for fold '+ str(i)+' exact match ratio is: ', exact_match_ratio)


Epoch 1/30
1013/1013 [==============================] - 83s 70ms/step - loss: 0.1819 - binary_accuracy: 0.9228 - val_loss: 0.1426 - val_binary_accuracy: 0.9379
Epoch 2/30
1013/1013 [==============================] - 73s 72ms/step - loss: 0.1186 - binary_accuracy: 0.9486 - val_loss: 0.1166 - val_binary_accuracy: 0.9499
Epoch 3/30
1013/1013 [==============================] - 75s 74ms/step - loss: 0.0961 - binary_accuracy: 0.9588 - val_loss: 0.1031 - val_binary_accuracy: 0.9571
Epoch 4/30
1013/1013 [==============================] - 75s 74ms/step - loss: 0.0831 - binary_accuracy: 0.9648 - val_loss: 0.0990 - val_binary_accuracy: 0.9595
Epoch 5/30
1013/1013 [==============================] - 75s 74ms/step - loss: 0.0742 - binary_accuracy: 0.9690 - val_loss: 0.0955 - val_binary_accuracy: 0.9615
Epoch 6/30
1013/1013 [==============================] - 75s 74ms/step - loss: 0.0673 - binary_accuracy: 0.9720 - val_loss: 0.0940 - val_binary_accuracy: 0.9623
Epoch 7/30
1013/1013 [==================

In [17]:
#9th fold
epochs = 30 # This should be at least 30 for convergence

fold_wise_binary_accuracy=[]
fold_wise_exact_match=[]

#for i in range(1):
for i in range(8, 9):
    model.compile(keras.optimizers.Adam(learning_rate=0.0001), loss="binary_crossentropy", metrics=["binary_accuracy"])

    model.fit(X_train[i],Y_train[i], epochs= epochs, validation_split=0.1)

    print('Model training done for fold '+str(i))
    predictions=model.predict(X_test[i])
    print('Predictions done for fold '+str(i))

    bins = np.array([0.5])
    count_exact_match=0
    sum_acc=0

    for j in range(len(predictions)):
        discrete_prediction=np.digitize(predictions[j], bins)
        acc=accuracy_score(Y_test[i][j],discrete_prediction)
        sum_acc=sum_acc+acc
        if acc==1:
            count_exact_match=count_exact_match+1
    avg_acc=sum_acc/len(predictions)
    exact_match_ratio=count_exact_match/len(predictions)

    fold_wise_binary_accuracy.append(avg_acc)
    fold_wise_exact_match.append(exact_match_ratio)

    print('for fold '+ str(i)+' binary accuracy is: ', avg_acc)
    print('for fold '+ str(i)+' exact match ratio is: ', exact_match_ratio)


Epoch 1/30
1013/1013 [==============================] - 78s 71ms/step - loss: 0.1689 - binary_accuracy: 0.9294 - val_loss: 0.1300 - val_binary_accuracy: 0.9459
Epoch 2/30
1013/1013 [==============================] - 69s 68ms/step - loss: 0.1098 - binary_accuracy: 0.9546 - val_loss: 0.1127 - val_binary_accuracy: 0.9528
Epoch 3/30
1013/1013 [==============================] - 71s 70ms/step - loss: 0.0908 - binary_accuracy: 0.9627 - val_loss: 0.1040 - val_binary_accuracy: 0.9567
Epoch 4/30
1013/1013 [==============================] - 69s 69ms/step - loss: 0.0789 - binary_accuracy: 0.9677 - val_loss: 0.0991 - val_binary_accuracy: 0.9608
Epoch 5/30
1013/1013 [==============================] - 69s 68ms/step - loss: 0.0705 - binary_accuracy: 0.9712 - val_loss: 0.0925 - val_binary_accuracy: 0.9635
Epoch 6/30
1013/1013 [==============================] - 72s 71ms/step - loss: 0.0642 - binary_accuracy: 0.9738 - val_loss: 0.0918 - val_binary_accuracy: 0.9643
Epoch 7/30
1013/1013 [==================